In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2020-11-30 07:25:52,139 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-30 07:25:52,140 - settings - Setting database.user to celiib
INFO - 2020-11-30 07:25:52,141 - settings - Setting database.password to newceliipass
INFO - 2020-11-30 07:25:52,147 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-30 07:25:52,148 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 07:25:52,161 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-30 07:25:52,360 - settings - Setting enable_python_native_blobs to True


# configuring the virtual module

In [4]:
test_mode = True

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-30 07:26:09,748 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-30 07:26:09,750 - settings - Setting database.user to celiib
INFO - 2020-11-30 07:26:09,751 - settings - Setting database.password to newceliipass
INFO - 2020-11-30 07:26:09,756 - settings - Setting enable_python_native_blobs to True


Sleeping 0 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2020-11-30 07:26:09,955 - settings - Setting enable_python_native_blobs to True


# Defining Our Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 8207


In [8]:
#so that it will have the adapter defined
from datajoint_utils import *

In [9]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [10]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """

#     key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0" & "segment_id=864691136309663834").proj())
#     key_source = (minnie.Decimation() & "n_faces>500000").proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
              & minnie.ManualProofreadIDs() & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()) - (minnie.ManualProofreadIDs() & minnie.Decomposition.proj()).proj()

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running The Populate

In [15]:
(minnie.schema.jobs & "table_name='__decomposition'") #& "status='error'"
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,1c541d6ff8c8608d4e216c92e7cba7a6,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,116269,2020-11-30 00:11:09
__decomposition,2c23e70348d74868b3be58f58279aafa,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,116266,2020-11-30 00:05:45
__decomposition,330c5ee82ec0229b6410ed1c7ad101c1,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,29,108695,2020-11-30 00:06:36
__decomposition,33829bd6b829f5c975edef606dc6848c,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,116258,2020-11-30 00:08:04
__decomposition,40a41dd78a30ef307b5c2f0c370dbbec,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,29,108703,2020-11-30 00:55:03
__decomposition,5518a878a77259264f59b2aded05b9da,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,116261,2020-11-30 00:13:41
__decomposition,57390a887b46d04d117cedcf57dedc9f,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,29,108988,2020-11-30 00:24:46
__decomposition,5c6d70eff1ec80e5e120d8d90cf0fec6,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,29,108996,2020-11-30 01:02:36
__decomposition,789d0240b106e9bebf30d3011acf25ea,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,30,116265,2020-11-30 00:03:07
__decomposition,9423a28c873d8ac837c7a9808bbb4faa,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,116271,2020-11-30 00:11:15


In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 900))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")